In [1]:
# dataset from huggingface
from datasets import load_dataset
dataset = load_dataset("lawcompany/KLAID", 'ljp')

Found cached dataset klaid (/Users/yunho/.cache/huggingface/datasets/lawcompany___klaid/ljp/1.0.0/170cd5bb1a0d9f3e383773bc69b51eb6a717918f91b682fe094492d865feaf4c)
100%|██████████| 1/1 [00:00<00:00, 24.62it/s]


In [2]:
# legal fact data
import pandas as pd

df = pd.DataFrame(dataset['train'])
legal_fact = df['fact'].values.tolist()
legal_fact

['피고인은 2018. 8. 9. 23:33경 술을 마신 상태로 경산시 사동에 있는 상호 불상의 식당에서부터 같은 동에 있는 부영5차 앞 삼거리까지 B 스타렉스 승용차를 운전한 다음 승용차 안에서 잠을 자던 중, 차량 운전자가 시동을 걸어 놓고 잠을 자고 있다는 112 신고를 받고 현장에 출동한 경산경찰서 C파출소 소속 경위 D으로부터 피고인의 입에서 술 냄새가 나고 보행이 비틀거리는 등 술에 취한 상태에서 운전하였다고 인정할 만한 상당한 이유가 있어 약 10분 동안 총 3회에 걸쳐 음주측정기에 입김을 불어 넣는 방법으로 음주측정에 응할 것을 요구받고도 정당한 사유 없이 이에 응하지 아니하였다.',
 '피고인은 2016. 3. 19. 10:16경 경북 칠곡군 왜관읍 왜관대교 앞 도로에서부터 같은 군 북삼읍 칠곡대로 78 효성실버요양원 앞 도로에 이르기까지 약 8km 구간에서 자동차운전면허를 받지 아니하고 B 봉고Ⅲ 화물차를 운전하였다.',
 '피고인은 2016. 10. 10 16:55경 평택시 오성면 복합화력발전소 앞 도로에서부터 평택시 안중읍 석정삼거리 앞 도로까지 약 12km 구간에서 자동차운전면허를 받지 아니하고 B 포터초장축더블캡 화물차를 운전하였다.',
 '피고인 A은 노동일에 종사 중이다. 피고인은 2017. 2. 2. 20:00경 부산 해운대구 B, C식당 앞 노상에서 술을 마시고 D의 포터차량을 타려다가 주변에 있던 E 일행이 음주운전을 하려 한다고 말을 건 일로 서로 시비가 되었다. 피고인은 피해자 E과 시비를 벌이다. 피해자를 밀치고, 인근 주차장에서 들고 온 약 1.5m 길이의 쇠막대기로 E의 머리와 어깨 부위를 4~5회 가량 내리쳐 피해자를 폭행하였다.',
 '피고인은 2015. 7. 2. 06:35경 부산 부산진구 B에 있는 C슈퍼 앞길에서 피해자 D(66세) 운행의 택시에서 피고인이 분실한 신용카드를 피해자가 찾아주지 않고 그냥 가려고 한다는 이유로 피해자에게 “이 개쌔끼야, 씹할 놈아, 카드가 와 없노”라고 욕설을 하면서 택시 앞을 

In [3]:
legal_fact_100 = legal_fact[:100]

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_length = 1024

[nltk_data] Downloading package punkt to /Users/yunho/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
inputs = ["summarize: " + legal_fact_100[0]]

inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
output = model.generate(inputs['input_ids'], max_length=256, num_beams=4, early_stopping=True, do_sample=True, min_length=30, top_k=50, top_p=0.95)
decoded = tokenizer.batch_decode(output, skip_special_tokens=True)
predicted = nltk.sent_tokenize(decoded[0])

print(predicted)

['피고인은 술에 취한 상태에서 운전하였다고 인정할 만한 상당한 이유가 있어 음주측정에 응할 것을 요구받았으나 정당한 사유 없이 응하지 않았다.']


- 어떻게 file 전체를 한 번에 처리할 수 있는가

- 어떻게 여러 input(fact + act)를 한 번에 처리할 수 있는가

In [9]:
inputs = ["summarize: " + fact for fact in legal_fact_100]

inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
output = model.generate(inputs['input_ids'], max_length=256, num_beams=4, early_stopping=True, do_sample=True, min_length=30, top_k=50, top_p=0.95)
decoded = tokenizer.batch_decode(output, skip_special_tokens=True)
predicted = [nltk.tokenize.sent_tokenize(text) for text in decoded]

print(predicted)ㅌ

: 

: 